## Question 1. Version of Pipenv (1 point)

### in command line... 
### pip install pipenv
### pipenv --version
### pipenv, version 2024.2.0


## Question 2. Sklearn hash

### run pipenv install scikitlearn==1.5.2
### copy hash from Pipfile.lock: sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445

## Question 3. Probability of subscription (1 point)

### wrote below script:

import pickle


client = {"job": "management", "duration": 400, "poutcome": "success"}


with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)


with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)


X = dv.transform(client)

prob = model.predict_proba(X)[0, 1]

print(prob)


## Output:

% python script.py
/opt/anaconda3/envs/ml-zoomcamp/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/ml-zoomcamp/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
0.7590966516879658

## Question 4. Probability of subscription (Flask) (1 point)

### Created webapp.py file with flask:

from flask import Flask, request, jsonify
import pickle


with open('model1.bin', 'rb') as f_in:
    model = pickle.load(f_in)

with open('dv.bin', 'rb') as f_in:
    dv = pickle.load(f_in)


app = Flask('model')


@app.route('/predict', methods = ['POST'])
def predict():
    client = request.get_json()
    X = dv.transform(client)
    y_pred = model.predict_proba(X)[0, 1]
    sub = y_pred >= 0.5

    result = {
        'sub_prob' : float(y_pred),
        'sub' : bool(sub)
    }

    return jsonify(result)


### run:
% gunicorn --bind 0.0.0.0:5555 webapp:app

### created separate request.py file:

import requests


url = 'http://localhost:5555/predict'

client = {"job": "student", "duration": 280, "poutcome": "failure"}
resp = requests.post(url, json=client).json()

print(resp)


## Output:

% python request.py
{'sub': False, 'sub_prob': 0.33480703475511053}

## Question 5. Size of base image

### ran below commands:

% docker pull svizor/zoomcamp-model:3.11.5-slim

## Output:

3.11.5-slim: Pulling from svizor/zoomcamp-model
a803e7c4b030: Pull complete 
bf3336e84c8e: Pull complete 
eb76b60fbb0c: Pull complete 
a2cee97f4fbd: Pull complete 
0358d4e17ae3: Pull complete 
fb37f8d7a667: Pull complete 
4e69cd59a5af: Pull complete 
Digest: sha256:15d61790363f892dfdef55f47b78feed751cb59704d47ea911df0ef3e9300c06
Status: Downloaded newer image for svizor/zoomcamp-model:3.11.5-slim
docker.io/svizor/zoomcamp-model:3.11.5-slim
(base) saumyasrivastava@SAUMYAs-Laptop Deployment % docker images
REPOSITORY              TAG           IMAGE ID       CREATED       SIZE
svizor/zoomcamp-model   3.11.5-slim   975e7bdca086   10 days ago   130MB
(base) saumyasrivastava@SAUMYAs-Laptop Deployment % ls
Pipfile         Pipfile.lock    dv.bin          model1.bin      request.py      script.py       webapp.py
(base) saumyasrivastava@SAUMYAs-Laptop Deployment % docker images
REPOSITORY              TAG           IMAGE ID       CREATED       SIZE
svizor/zoomcamp-model   3.11.5-slim   975e7bdca086   10 days ago   130MB

## Question 6. Probability of subscription (Docker)

### Created Docker File:

FROM python:3.11.5-slim

RUN pip install pipenv

WORKDIR /app                                                                

COPY ["Pipfile", "Pipfile.lock", "./"]

RUN pipenv install --system --deploy

COPY ["webapp.py", "model1.bin", "dv.bin", "./"]

EXPOSE 5555

ENTRYPOINT ["gunicorn", "--bind", "0.0.0.0:5555", "webapp:app"]


### Ran below commands:

% docker build -t webapp .                      
[+] Building 3.9s (12/12) FINISHED                                                                                                             docker:desktop-linux
 => [internal] load build definition from Dockerfile                                                                                                           0.0s
 => => transferring dockerfile: 408B                                                                                                                           0.0s
 => [internal] load metadata for docker.io/library/python:3.11.5-slim                                                                                          3.9s
 => [auth] library/python:pull token for registry-1.docker.io                                                                                                  0.0s
 => [internal] load .dockerignore                                                                                                                              0.0s
 => => transferring context: 2B                                                                                                                                0.0s
 => [internal] load build context                                                                                                                              0.0s
 => => transferring context: 828B                                                                                                                              0.0s
 => [1/6] FROM docker.io/library/python:3.11.5-slim@sha256:edaf703dce209d774af3ff768fc92b1e3b60261e7602126276f9ceb0e3a96874                                    0.0s
 => CACHED [2/6] RUN pip install pipenv                                                                                                                        0.0s
 => CACHED [3/6] WORKDIR /app                                                                                                                                  0.0s
 => CACHED [4/6] COPY [Pipfile, Pipfile.lock, ./]                                                                                                              0.0s
 => CACHED [5/6] RUN pipenv install --system --deploy                                                                                                          0.0s
 => CACHED [6/6] COPY [webapp.py, model1.bin, dv.bin, ./]                                                                                                      0.0s
 => exporting to image                                                                                                                                         0.0s
 => => exporting layers                                                                                                                                        0.0s
 => => writing image sha256:eed6d7c67d77e057f130243d067bf94d16bb73d3347ca00b22651be2478f9c60                                                                   0.0s
 => => naming to docker.io/library/webapp                                                                                                                      0.0s

View build details: docker-desktop://dashboard/build/desktop-linux/desktop-linux/2j8aypwz3z2qcjyy1eknrde13
(base) saumyasrivastava@SAUMYAs-Laptop Deployment % docker run -it -p 5555:5555 webapp:latest
[2024-10-29 12:37:51 +0000] [1] [INFO] Starting gunicorn 23.0.0
[2024-10-29 12:37:51 +0000] [1] [INFO] Listening at: http://0.0.0.0:5555 (1)
[2024-10-29 12:37:51 +0000] [1] [INFO] Using worker: sync
[2024-10-29 12:37:51 +0000] [7] [INFO] Booting worker with pid: 7


### changed request.py file as:

import requests


url = 'http://localhost:5555/predict'

client = {"job": "management", "duration": 400, "poutcome": "success"}
resp = requests.post(url, json=client).json()

print(resp)


### ran in separate cmd:

% python request.py

## Output:

{'sub': True, 'sub_prob': 0.7590966516879658}